In [1]:
# coding: utf-8
from __future__ import absolute_import, division, print_function

import os
import time
import os.path as osp

import tensorflow as tf
import tensorflow.contrib.eager as tfe

import mnist_dataset
from easydict import EasyDict as edict

tf.enable_eager_execution()

ImportError: Traceback (most recent call last):
  File "/home/zz/.local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/zz/.local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/zz/.local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

## 损失函数

In [2]:
def loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
    )

为什么要自己重写一个这么复杂的compute_accuracy?

In [3]:
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1, output_type=tf.int64)
    labels = tf.cast(labels, tf.int64)
    batch_size = int(logits.shape[0])
    return tf.reduce_sum(tf.case(tf.equal(predictions, labels), dtype=tf.float32)) / batch_size

train函数也写的好麻烦。用了很多tf.contrib的东西，感觉不妥。

In [4]:
def train(model, optimizer, dataset, step_counter, log_interval=None):
    start = time.time()
    for (batch, (images, labels)) in enumerate(tfe.Iterator(dataset)):
        with tf.contrib.summary.record_summaries_every_n_global_steps(10, global_step=step_counter):
            logits = model(images, training=True)
            loss_value = loss(logits, label)
            tf.contrib.summary.scalar('loss', loss_value)
            tf.contrib.summary.scalar('accuracy', compute_accuracy(logits, labels))
        grads = tape.gradient(loss_value, model.variables)
        optimizer.apply_gradients(
            zip(grads, model.variables), global_step=step_counter
        )
        if log_interval and batch%log_interval==0:
            rate = log_interval / (time.time() - start)
            print('Step #%d\tLoss: %.6f (%d step/sec)' % (batch, loss_value, rate))
            start = time.time()

## 运行训练

In [5]:
def get_opt():
    opt = edict()
    opt.log_interval = 10
    opt.output_dir= 'output'
    opt.learning_rate=0.01
    opt.momentum=0.5
    opt.data_dir='/tmp/tensorflow/mnist/input_data'
    opt.model_dir='/tmp/tensorflow/mnist/checkpoints/'
    opt.batch_size=100
    opt.train_epochs=10
    
    return opt

In [6]:
def create_model(data_format):
    if data_format == 'channels_first':
        input_shape = [1, 28, 28]
    else:
        assert data_format == 'channels_last'
        input_shape = [28, 28, 1]
    
    tkl = tf.keras.layers
    max_pool = tkl.MaxPooling2D(
        (2, 2), (2, 2), padding='same', data_format=data_format
    )
    model = tf.keras.Sequential([
        tkl.Reshape(target_shape=input_shape, input_shape=(28*28,)),
        tkl.Conv2D(32, 5, padding='same', data_format=data_format, activation=tf.nn.relu),
        max_pool,
        tkl.Conv2D(64, 5, padding='same', data_format=data_format, activation=tf.nn.relu),
        max_pool,
        tkl.Flatten(),
        tkl.Dense(1024, activation=tf.nn.relu),
        tkl.Dropout(0.4),
        tkl.Dense(10)
    ])
    
    return model

In [7]:
# load data
opt = get_opt()
device = '/gpu:0'
data_format = 'channels_first'
train_dataset = mnist_dataset.train(opt.data_dir).shuffle(60000).batch(opt.batch_size)
test_dataset = mnist_dataset.test(opt.data_dir).batch(opt.batch_size)

# create model and optimizer
model = create_model(data_format)
optimizer = tf.train.MomentumOptimizer(opt.learning_rate, opt.momentum)

In [8]:
train_dir = osp.join(opt.output_dir, 'train')
test_dir = osp.join(opt.output_dir, 'eval')
tf.gfile.MakeDirs(opt.output_dir)
summary_writer = tf.contrib.summary.create_file_writer(
    train_dir, flush_millis=10000)
test_summary_writer = tf.contrib.summary.create_file_writer(
    test_dir, flush_millis=10000, name='test')

checkpoint_prefix = osp.join(opt.model_dir, 'ckpt')
step_counter = tf.train.get_or_create_global_step()
checkpoint = tfe.Checkpoint(model=model, optimizer=optimizer, step_counter=step_counter)

In [13]:
# 垃圾tensorflow，非得要设定环境变量CUDA_VISIBLE_DEVICES才能用GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4,5,6,7'
#with tf.device('/gpu:0'): #不行
with tf.device('/device:GPU:2'):
    #with tf.device(device):
    for _ in range(opt.train_epochs):
        start = time.time()
        with summary_writer.as_default():
            train(model, optimizer, train_dataset, step_counter, opt.log_interval)
        end = time.time()
        print('\nTrain time for epoch #%d (%d total steps): %f' % (checkpoint.save_counter.numpy()+1, step_counter.numpy(), end-start))
        with test_summary_writer.as_default():
            test(model, test_dataset)
        checkpoint.save(checkpoint_prefix)

InvalidArgumentError: /job:localhost/replica:0/task:0/device:GPU:2 unknown device.